# Coursera_Capstone_Project - Segmenting and Clustering Neighborhoods in Toronto

In [36]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

In [57]:
x = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup=BeautifulSoup(x, 'html.parser')

In [63]:
#find table
tables=soup.find('table')

In [64]:
#find rows
rows=tables.find_all('tr')

In [69]:
a=[]
b=[]
c=[]
#find data in each row 
for row in rows:
    value=row.find_all('td')
    if(len(value)>0):
        a.append(value[0].text.rstrip('\n'))
        b.append(value[1].text.rstrip('\n'))
        c.append(value[2].text.rstrip('\n'))
        

In [70]:
df=pd.DataFrame({"PostalCode":a,"Borough": b,"Neighborhood": c})
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [71]:
#drop cells of not assigned
df1 = df[df.Borough != "Not assigned"].reset_index(drop=True)
df1.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [72]:
# group neighborhoods
df2 = df1.groupby(["PostalCode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
df2.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [74]:
#to check the Postalcodes given in the question
x = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

column_names = ["PostalCode", "Borough", "Neighborhood"]
test_df = pd.DataFrame(columns=column_names)

for postcode in x:
    test_df = test_df.append(df2[df2["PostalCode"]==postcode], ignore_index=True)
    
test_df

,PostalCode,Borough,Neighborhood
0,M5G,Downtown Toronto,Central Bay Street
1,M2H,North York,Hillcrest Village
2,M4B,East York,"Parkview Hill, Woodbine Gardens"
3,M1J,Scarborough,Scarborough Village
4,M4G,East York,Leaside
5,M4M,East Toronto,Studio District
6,M1R,Scarborough,"Wexford, Maryvale"
7,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."
8,M9L,North York,Humber Summit
9,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har..."
